Tools - interfaces used by an LLM, agent or chain can use to interact with the world
* GPT 4o came in Dec 2023. So the data is outdated. 
* Integrate the app with tools - ARXIV has research papers for example, Wikipedia etc.
* We can create our own tool. eg: doc Q&A
* The response may not be there in the LLM model. It will interact with the tools and get us the response

To interact with tools, we need Agents
* CREW AI - uses multiple tools and agents
* Uses a language model to choose a sequence of actions
  
Good number of tools are provided by langchain


In [3]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [ ]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=5000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

In [27]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=5000)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [29]:
tools = [arxiv, wiki]

In [30]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [31]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()


In [32]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any info about langsmith")

In [ ]:
tools = [wiki, arxiv, retriever_tool]
tools

In [34]:
### Tools and LLM Models using Agent executor
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()


llm = ChatGroq(groq_api_key=os.getenv("GROQ_API_KEY"), model="Llama3-8b-8192")

In [35]:
## Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [36]:
## To run the above we need agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [37]:
from langchain.agents import AgentExecutor
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [38]:
executor.invoke({'input':'Tell me about langsmith'})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith






Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation neede

{'input': 'Tell me about langsmith',
 'output': "Based on the output from the tool, it seems that LangSmith is a platform for building production-grade Large Language Model (LLM) applications. It allows users to closely monitor and evaluate their applications, making it easier to ship quickly and with confidence.\n\nSome of the key features of LangSmith include:\n\n* Close monitoring and evaluation of LLM applications\n* Integration with LangChain's open-source frameworks langchain and langgraph\n* Ability to use the platform with any provider of choice, not just OpenAI\n* SDKs available for Python and JavaScript/TypeScript\n* Observability tutorials and guides to help users get started\n\nLangSmith also provides a user interface for exploring the results of evaluations and tracing, as well as a community forum and documentation for further information."}

In [39]:
executor.invoke({'input':'What is machine learning?'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.
Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning. 
From a theoretical viewpoint, p

{'input': 'What is machine learning?',
 'output': 'Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.'}